<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://raw.githubusercontent.com/Unidata/MetPy/master/src/metpy/plots/_static/unidata_150x150.png" alt="Unidata Logo" style="height: 98px;">
</div>

<h1>Declarative Syntax Example</h1>
<h3>Unidata AMS 2021 Student Conference</h3>

<div style="clear:both"></div>
</div>

---

<div style="float:right; width:1200 px"><img src="../../instructors/images/declarative_surface.png" alt="sin(x) with few data points" style="height: 300px;"></div>


### Focuses
* Read in satellite data using Siphon
* Plot satellite, surface, and upper air observations using the declarative interface
* Utilize MetPy's METAR functionality to load recent surface observations
* 
* Calculate $y = f(x) = sin(x)$ with [NumPy](https://numpy.org/doc/stable/) (don't forget math text!)
* Demonstrate visualizing this with [Matplotlib](https://matplotlib.org/)


### Objectives
1. [Satellite Data](#1.-Satellite-Observations)
1. [Surface Observations](#2.-Surface-Observations)
1. [Upper-air Observations](#3.-Upper-air-observations)
1. [Model Data](#4.-Model-data)
1. [Final reminders and instructions for instructors](#Final-reminders)

---

The MetPy declarative syntax, similar to GEMPAK, allows for a simplified interface to create common meteorological analyses including surface observation plots

### Imports

In [ ]:
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings("ignore")

import cartopy.crs as ccrs
import pandas as pd

from metpy.cbook import get_test_data
from metpy.io import metar
import metpy.plots as mpplots
from metpy.units import units
from siphon.catalog import TDSCatalog

## 1. Satellite Observations

### Accessing data via Siphon


In [ ]:
# Create variables for URL generation
image_date = datetime.utcnow().date()
region = 'CONUS'
channel = 8

# Create the URL to provide to siphon
data_url =  ('https://thredds.ucar.edu/thredds/catalog/satellite/goes/east/products/'
            f'CloudAndMoistureImagery/{region}/Channel{channel:02d}/'
            f'{image_date:%Y%m%d}/catalog.xml')

# Create a catalog for the dataset
cat = TDSCatalog(data_url)

In [ ]:
# Read the second dataset and print the output
dataset = cat.datasets[1]
print(dataset)

In [ ]:
# Now we read it in using xarray
ds = dataset.remote_access(use_xarray=True)

### Plotting Satellite Data



To plot our data we'll be using MetPy's new declarative plotting functionality. You can write lots of matplotlib based code, but this interface greatly reduces the number of lines you need to write to get a great starting plot and then lets you customize it. The declarative plotting interface consists of three fundamental objects/concepts:

* **Plot** - This is the actual representation of the data and can be ImagePlot, ContourPlot, Plot2D, FilledContourPlot and BarbPlot.
* **Panel** - This is a single panel (i.e. coordinate system). Panels contain plots. Currently the MapPanel is the only panel type available.
* **Panel Container** - The container can hold multiple panels to make a multi-pane figure. Panel Containers can be thought of as the whole figure object in matplotlib.

So containers have panels which have plots. It takes a second to get that straight in your mind, but it makes setting up complex figures very simple.

For this plot we need a single panel and we want to plot the satellite image, so we'll use the `ImagePlot`.


In [ ]:
img = mpplots.ImagePlot()
img.data = ds
img.field = 'Sectorized_CMI'

Next, we'll make the panel that our image will go into, the MapPanel object and add the image to the plots on the panel.

In [ ]:
panel = mpplots.MapPanel()
panel.plots = [img]

Finally, we make the PanelContainer and add the panel to its container. Remember that since we can have multiple plots on a panel and multiple panels on a plot, we use lists. In this case is just happens to be a list of length 1.

In [ ]:
pc = mpplots.PanelContainer()
pc.panels = [panel]

Unlike working with matplotlib directly in the notebooks, this figure hasn't actually been rendered yet. Calling the show method of the panel container builds up everything, renders, and shows it to us.

In [ ]:
pc.show()

Now, let's add a few more labels/colors onto there and include this whole process in a single cell

In [ ]:
# Make the image plot
img = mpplots.ImagePlot()
img.data = ds
img.field = 'Sectorized_CMI'
img.colormap = 'WVCIMSS_r'
img.image_range = (195, 265)

# Make the map panel and add the image to it
panel = mpplots.MapPanel()
panel.layers = ['coastline', 'borders', 'states']
panel.plots = [img]

# Make the panel container and add the panel to it
pc = mpplots.PanelContainer()
pc.panels = [panel]
pc.size=(15, 9)

# Add a timestamp
start_time = datetime.strptime(ds.start_date_time, '%Y%j%H%M%S')
mpplots.add_timestamp(panel.ax, time=start_time)

# Show the plot
pc.show()

---
## 2. Surface Observations

### Bring in surface data
In this example, data is originally from the Iowa State ASOS archive (https://mesonet.agron.iastate.edu/request/download.phtml) downloaded through a separate Python script. The data are pre-processed to determine sky cover and weather symbols from text output.

In [ ]:
data = pd.read_csv(get_test_data('SFC_obs.csv', as_file_obj=False),
                   infer_datetime_format=True, parse_dates=['valid'])

### Plotting the surface data

Use the declarative plotting interface to plot surface observations over the state of Georgia.

In [ ]:
# Plotting the Observations using a 15 minute time window for surface observations
obs = mpplots.PlotObs()
obs.data = data
obs.time = datetime(1993, 3, 12, 13)
obs.time_window = timedelta(minutes=15)
obs.level = None
obs.fields = ['tmpf', 'dwpf', 'emsl', 'cloud_cover', 'wxsym']
obs.locations = ['NW', 'SW', 'NE', 'C', 'W']
obs.colors = ['red', 'green', 'black', 'black', 'blue']
obs.formats = [None, None, lambda v: format(10 * v, '.0f')[-3:], 'sky_cover',
               'current_weather']
obs.vector_field = ('uwind', 'vwind')
obs.reduce_points = 1

# Add map features for the particular panel
panel = mpplots.MapPanel()
panel.layout = (1, 1, 1)
panel.area = 'ga'
panel.projection = ccrs.PlateCarree()
panel.layers = ['coastline', 'borders', 'states']
panel.plots = [obs]

# Collecting panels for complete figure
pc = mpplots.PanelContainer()
pc.size = (10, 10)
pc.panels = [panel]

# Showing the results
pc.show()

### Working with METAR data

One can also use the realtime METARs as a datasource, utilizing MetPy's METAR parsing functionality

Data is from the [Unidata Thredds Test Server](https://thredds-test.unidata.ucar.edu/thredds/catalog/noaaport/text/metar/catalog.html) which includes data from thirty days in the past to current observations

Another data source is the Iowa State ASOS Archive, described previously

In [ ]:
# Use siphon to read in a METAR text file from the Unidata Thredds server
cat = TDSCatalog('https://thredds-test.unidata.ucar.edu/thredds/catalog/noaaport/text/metar/catalog.xml')

# Load in the latest hour's data, subtracting 30 minutes to ensure the observations have enough time to be read
metar_ds = cat.datasets.filter_time_nearest(datetime.utcnow() - timedelta(minutes=30))

# Download the file
metar_ds.download()

# Parse the text file and create a Pandas dataframe
sfc_data = metar.parse_metar_file(metar_ds.name)

Next, we want to make sure we convert the temperature and dewpoint from degrees Celsius to degrees Fahrenheit

In [ ]:
sfc_data['air_temperature']  = (sfc_data['air_temperature'].values * units(sfc_data.units['air_temperature'])).to('degF')
sfc_data['dew_point_temperature'] = (sfc_data['dew_point_temperature'].values * units(sfc_data.units['dew_point_temperature'])).to('degF')

Now we setup our `PlotObs` object

In [ ]:
# Setup the PlotObs
obs = mpplots.PlotObs()
obs.data = sfc_data
obs.level = None
obs.fields = ['air_temperature', 'dew_point_temperature', 'air_pressure_at_sea_level', 'cloud_coverage', 'present_weather', 'past_weather']
obs.locations = ['NW', 'SW', 'NE', 'C', 'W', 'SE']
obs.formats = [None, None, lambda v: format(10 * v, '.0f')[-3:], 'sky_cover',
               'current_weather', 'current_weather']
obs.vector_field = ('eastward_wind', 'northward_wind')
obs.reduce_points = 2

# Setup the MapPanel
panel = mpplots.MapPanel()
panel.layers = ['coastline', 'borders', 'states', 'land', 'ocean']
panel.area = 'centus'
panel.projection = 'lcc'
panel.plots = [obs]
#panel.title = 'Surface Observations'

pc = mpplots.PanelContainer()
pc.panels = [panel]
pc.size=(15, 9)
mpplots.add_timestamp(panel.ax, time=start_time)
pc.show()

---
## 3. Upper-air observations

Plot upper level data using test data - the test data was pulled from the Iowa State upper-air archive which also requires preprocessing to add lats/lons. For now, we will use the test data from MetPy

In [ ]:
data = pd.read_csv(get_test_data('UPA_obs.csv', as_file_obj=False))

In [ ]:
data

In [ ]:
# Plotting the Observations
obs = mpplots.PlotObs()
obs.data = data
obs.time = datetime(1993, 3, 14, 0)
obs.level = 500 * units.hPa
obs.fields = ['temperature', 'dewpoint', 'height']
obs.locations = ['NW', 'SW', 'NE']
obs.formats = [None, None, lambda v: format(v, '.0f')[:3]]
obs.vector_field = ('u_wind', 'v_wind')
obs.reduce_points = 0

# Add map features for the particular panel
panel = mpplots.MapPanel()
panel.layout = (1, 1, 1)
panel.area = (-124, -72, 20, 53)
panel.projection = 'lcc'
panel.layers = ['coastline', 'borders', 'states', 'land', 'ocean']
panel.plots = [obs]

# Collecting panels for complete figure
pc = mpplots.PanelContainer()
pc.size = (15, 10)
pc.panels = [panel]

# Showing the results
pc.show()

## 4. Model data

Read in most recent NAM data from the Unidata Thredds Server Other model data can be found using the following website https://thredds.ucar.edu/thredds/catalog.html

In [ ]:
# Import data and remotely access it via Siphon using xarray
cat_string = 'https://thredds.ucar.edu/thredds/catalog/grib/NCEP/NAM/CONUS_12km/latest.xml'
cat = TDSCatalog(cat_string)
model_ds = cat.datasets[0].remote_access(use_xarray=True).metpy.parse_cf()

In [ ]:
# Select the first time
model_ds = model_ds.sel(time = model_ds.time[0:1])

In [ ]:
# Convert data to proper units using MetPy conversion tools
model_ds['Pressure_reduced_to_MSL_msl'] = model_ds.Pressure_reduced_to_MSL_msl.metpy.convert_units('hPa')
model_ds['Temperature_height_above_ground'] = model_ds.Temperature_height_above_ground.metpy.convert_units('degF')

Smooth the data using gaussian smoothing from SciPy

In [ ]:
from scipy.ndimage import gaussian_filter

# Smooth contour for MSLP
model_ds['Pressure_reduced_to_MSL_msl'] = (['time1', 'y','x'], gaussian_filter(model_ds['Pressure_reduced_to_MSL_msl'], sigma=3.0))

# Smooth contour for surface temperature
model_ds['Temperature_height_above_ground'] = (['time1', 'height_above_ground1','y','x'], gaussian_filter(model_ds['Temperature_height_above_ground'], sigma=3.0))

In [ ]:
# Setup the ContourPlot, setting the desired variable to plot
contour_mslp = mpplots.ContourPlot()
contour_mslp.data = model_ds
contour_mslp.field = 'Pressure_reduced_to_MSL_msl' # Variable within the xarray dataset that will be plotted
contour_mslp.time = datetime.utcnow()
contour_mslp.level = None
contour_mslp.linecolor = 'black'
contour_mslp.clabels = True
contour_mslp.contours = list(range(900, 1100, 4)) # Contour intervals

# Setup another ContourPlot for temperature (in degrees Fahrenheit)
contour_temp = mpplots.ContourPlot()
contour_temp.data = model_ds
contour_temp.field = 'Temperature_height_above_ground'
contour_temp.time = datetime.utcnow()
contour_temp.linecolor = 'red'
contour_temp.linestyle = 'dashed'
contour_temp.clabels = True
contour_temp.contours = list(range(-50, 100, 10)) # Contour intervals

# Plot the east coast 
panel = mpplots.MapPanel()
panel.area = 'east'
panel.layers = ['coastline', 'borders', 'states', 'land', 'ocean']
panel.plots = [contour_temp, contour_mslp]

pc = mpplots.PanelContainer()
pc.panels = [panel]
pc.size=(15, 9)
mpplots.add_timestamp(panel.ax, time=datetime.utcnow())
pc.show()

Now try using a `FilledContourPlot`

In [ ]:
cfill = mpplots.FilledContourPlot()
cfill.time = datetime.utcnow()
cfill.data = model_ds
cfill.field = 'Temperature_isobaric'
cfill.level = 850 * units.hPa
cfill.colormap = 'RdBu_r'
cfill.contours = list(range(230, 320, 2))
cfill.colorbar = 'vertical'

# Plot the east coast 
panel = mpplots.MapPanel()
panel.area = 'east'
panel.layers = ['coastline', 'borders', 'states']
panel.plots = [cfill]

pc = mpplots.PanelContainer()
pc.panels = [panel]
pc.size=(15, 9)
mpplots.add_timestamp(panel.ax, time=start_time)
pc.show()

We can also add wind barbs on there using `BarbPlot`

In [ ]:
# Plot wind barbs
barb = mpplots.BarbPlot()
barb.time = datetime.utcnow()
barb.data = model_ds
barb.level = 850 * units.hPa
barb.field = ['u-component_of_wind_isobaric', 'v-component_of_wind_isobaric']
barb.skip = (15, 15)
barb.color = 'black'
barb.barblength = 6.5
barb.earth_relative = False

# Use the same colorfill from the cells above
cfill = mpplots.FilledContourPlot()
cfill.time = datetime.utcnow()
cfill.data = model_ds
cfill.field = 'Temperature_isobaric'
cfill.level = 850 * units.hPa
cfill.colormap = 'RdBu_r'
cfill.contours = list(range(230, 320, 2))
cfill.colorbar = 'vertical'

# Plot the east coast 
panel = mpplots.MapPanel()
panel.area = 'east'
panel.layers = ['coastline', 'borders', 'states']
panel.plots = [cfill, barb]

pc = mpplots.PanelContainer()
pc.panels = [panel]
pc.size = (15, 9)
mpplots.add_timestamp(panel.ax, time=start_time)
pc.show()